In [1]:
import pandas as pd
from dotenv import load_dotenv
import os
from api import get_token, get_camera_token
from pyroclient import Client
import glob
from PIL import Image
import io
from utils import dl_data

In [2]:
API_URL = "http://0.0.0.0:5050"
load_dotenv("../.env")
SUPERADMIN_LOGIN = os.environ.get("SUPERADMIN_LOGIN")
SUPERADMIN_PWD = os.environ.get("SUPERADMIN_PWD")

# Get access token
admin_access_token = get_token(API_URL, SUPERADMIN_LOGIN, SUPERADMIN_PWD)


In [3]:
users = pd.read_csv("../data/csv/API_DATA_DEV - users.csv")
cameras = pd.read_csv("../data/csv/API_DATA_DEV - cameras.csv")
users

,id,organization_id,password,login,role,created_at
0,2,2,test,test77,agent,2024-02-23T08:18:45.447773
1,3,3,test,test07,agent,2024-02-23T08:18:45.447773


In [4]:
user = "test77"
organization_id = users[users["login"]==user]["organization_id"].item()

user_cameras = cameras[cameras["organization_id"]==organization_id]
user_cameras

,id,organization_id,name,angle_of_view,elevation,lat,lon,is_trustable,last_active_at,created_at
0,2,2,videlles-01,54.2,110,48.4783,2.4242,True,NaN,2023-11-07T15:07:19.226673
1,3,2,videlles-02,54.2,110,48.4783,2.4242,True,NaN,2023-11-07T15:07:19.226673
2,4,2,croix-augas-01,54.2,110,48.4267,2.7109,True,NaN,2023-11-07T15:07:19.226673
3,5,2,croix-augas-02,54.2,110,48.4267,2.7109,True,NaN,2023-11-07T15:07:19.226673
4,6,2,moret-sur-loing-01,54.2,110,48.3792,2.8208,True,NaN,2023-11-07T15:07:19.226673
5,7,2,moret-sur-loing-02,54.2,110,48.3792,2.8208,True,NaN,2023-11-07T15:07:19.226673
6,8,2,nemours-01,54.2,110,48.2605,2.7064,True,NaN,2023-11-07T15:07:19.226673
7,9,2,nemours-02,54.2,110,48.2605,2.7064,True,NaN,2023-11-07T15:07:19.226673


In [5]:
send_alert_from_cam_ids = [2, 4, 5, 7] # select cameras

In [6]:
if not os.path.isdir("selection-true-positives"):
    dl_data()

In [7]:
sequances_folders = glob.glob("selection-true-positives/*")
len(sequances_folders)

12

In [8]:
# Heartbeat

for camera_id in send_alert_from_cam_ids:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)

    camera_client.heartbeat()

    sequances_folder = sequances_folders[camera_id]

    

In [9]:
# Update last image

for camera_id in send_alert_from_cam_ids:
    
    camera_token = get_camera_token(API_URL, camera_id, admin_access_token)
    camera_client  = Client(camera_token, API_URL)


    sequances_folder = sequances_folders[camera_id]
    imgs = glob.glob(f"{sequances_folder}/images/*")
    img_file = imgs[0]

    stream = io.BytesIO()
    im = Image.open(img_file)
    im.save(stream, format="JPEG", quality=80)
    response = camera_client.update_last_image(stream.getvalue())
    print(response.text)



    

{"name":"videlles-02","angle_of_view":54.2,"is_trustable":true,"last_image":"2-20250711145724-184321c7.jpg","organization_id":2,"id":2,"elevation":110.0,"lat":48.4783,"lon":2.4242,"last_active_at":"2025-07-11T14:57:24.531372","created_at":"2025-07-11T12:58:08.361140"}
{"name":"croix-augas-02","angle_of_view":54.2,"is_trustable":true,"last_image":"4-20250711145724-a84c05a8.jpg","organization_id":2,"id":4,"elevation":110.0,"lat":48.4267,"lon":2.7109,"last_active_at":"2025-07-11T14:57:24.571574","created_at":"2025-07-11T12:58:08.377063"}
{"name":"moret-sur-loing-01","angle_of_view":54.2,"is_trustable":true,"last_image":"5-20250711145724-d279d770.jpg","organization_id":2,"id":5,"elevation":110.0,"lat":48.3792,"lon":2.8208,"last_active_at":"2025-07-11T14:57:24.617543","created_at":"2025-07-11T12:58:08.404154"}
{"name":"nemours-01","angle_of_view":54.2,"is_trustable":true,"last_image":"7-20250711145724-0c0072b6.jpg","organization_id":2,"id":7,"elevation":110.0,"lat":48.2605,"lon":2.7064,"las